# World Data League 2021
## Notebook Template

This notebook is one of the mandatory deliverables when you submit your solution (alongside the video pitch). Its structure follows the WDL evaluation criteria and it has dedicated cells where you can add descriptions. Make sure your code is readable as it will be the only technical support the jury will have to evaluate your work.

The notebook must:

*   💻 have all the code that you want the jury to evaluate
*   🧱 follow the predefined structure
*   📄 have markdown descriptions where you find necessary
*   👀 be saved with all the output that you want the jury to see
*   🏃‍♂️ be runnable


## Using R in ipynb notebooks

**Note that the script is written in R, therefore in order to be able to execute it, R kernel should be installed and selected in Jupyter.**

See more at https://docs.anaconda.com/anaconda/user-guide/tasks/using-r-language/ and a practical guide at https://kyleake.medium.com/how-to-install-r-in-jupyter-with-irkernel-in-3-steps-917519326e41.

Also note that *geojsonio* package is not enlisted among the packages of the [Anaconda repo](https://repo.anaconda.com/pkgs/r/), but can still be installed through R. Unfortunately the other R packages for geojson manipulation could not handle the specific object (sp SpatialMultiPointsDataFrame) correctly.

In [ ]:
#uncomment to run
#install.packages("geojsonio")

## External links and resources

There is a variety of sites publishing open data in Waterloo (CA) area:
- [Waterloo city](https://data.waterloo.ca/datasets/75fa68b17e5b421d9d881f987c2d43fc_0?geometry=-81.052%2C43.339%2C-79.893%2C43.514)
- [Kitchener city](https://open-kitchenergis.opendata.arcgis.com/datasets/traffic-collisions?geometry=-80.994%2C43.339%2C-79.951%2C43.514)
- [Waterloo region](https://rowopendata-rmw.opendata.arcgis.com/datasets/KitchenerGIS::traffic-collisions?geometry=-80.994%2C43.339%2C-79.951%2C43.514)

All of these sites offer in theory the same dataset, **Traffic Collisions in Kitchener**. The **API link for the full geojson data** is 
https://opendata.arcgis.com/datasets/75fa68b17e5b421d9d881f987c2d43fc_0.geojson from all 3 sites.

We download another dataset from Waterloo Open Data Portal on the **Kitchener road system** (look for API at https://data.waterloo.ca/datasets/KitchenerGIS::roads), and also experimented with **OpenStreetMap** data through the *osmdata* R package.

**Note:** we met a disturbing phenomenon, which may also affect the execution of this notebook, namely that occasionally the downloaded dataset consists of not 12.201, but 4.099 rows (and it is not a subset of the bigger dataset!). We do not understand this (even asked for help through the wdl-challenge-support channel on 29/04/2021), but it can impact the findings of this notebook.

## Introduction

We chose the ***Patterns and predictive modelling of traffic accidents*** in stage 2 of WDL.  The goal of our submission is to visualize spatial patterns and apply predictive modelling techniques to help better understand factors behind traffic accidents. 

We aim to provide an explainable predictive model of traffic accidents at street (road segment) level by the period of the day. 

Our steps were:

 - Load, understand and enrich data.

 - Understand potential use cases for our model. This will help with the decision making process while modeling.

 - Start building the groundwork for the model. Some, otherwise useful data didn't support our predictive model. As we might have been able to incorporate these in the analysis if we had more time, we'll still briefly introduce these.

 - Develop models

 - Identify key variables and suggest actionable proposal which can offer a good return on investment for the community.


### Potential Use Cases

 Understanding and predicting traffic accidents can have the following main use cases:

 - Identify and justify road (re)construction projects (e.g. change intersections or add additional protective devices) with the highest returns
 
 - Support the design of the development of the road network of a city

 - Support proactive placement of traffic police

 - Adjust traffic signs and adaptive speed limit control

 - Predict insurance claims

 - Target certain demographic groups to reduce accidents (improve safety for bicycles, reduce drinking and driving, reduce mobile usage while driving)

## Development

In [ ]:
url_geojson <- "https://opendata.arcgis.com/datasets/75fa68b17e5b421d9d881f987c2d43fc_0.geojson" 
url_geojson_roads <- "https://opendata.arcgis.com/datasets/c2a28a64955943ffad68e7ce099c5556_0.geojson"

In [ ]:
require(tidyverse)
require(leaflet)
require(leaflet.extras)
require(rgdal)
require(sf)
require(osmdata)
require(ggmap)
require(lubridate)
require(sp)
require(rgeos)
require(raster)
require(weathercan)
require(naniar)
require(maptools)
require(data.table)
require(RcmdrMisc)
require(caret)
require(RColorBrewer)
require(skimr)
require(gridExtra)

In [ ]:
# the data is downloaded as an sp SpatialMultiPointsDataFrame object, which is designed to manipulate point set data. 
# this is unnecessarily complex for our use case and the DF contains only coordinates of 1 point in each row
# so the data is turned to SpatialPointsDataFrame object
url_geojson <- URLencode(url_geojson)
data_waterloo1 <- geojsonio::geojson_read(url_geojson, what = "sp")
longlat <- data.frame(lon = coordinates(data_waterloo1)[,1], lat = coordinates(data_waterloo1)[,2])
data_waterloo <- SpatialPointsDataFrame(coords = longlat, data = data_waterloo1@data, proj4string = CRS("+proj=longlat +datum=WGS84 +no_defs"))
names(data_waterloo) <- tolower(names(data_waterloo))

In [ ]:
url_geojson_roads <- URLencode(url_geojson_roads)
data_waterloo_roads <- geojsonio::geojson_read(url_geojson_roads, what = "sp")   
names(data_waterloo_roads) <- tolower(names(data_waterloo_roads))

### Example OpenStreetMap (OSM)  application in R
Many features could be downloaded using the *osmdata* R package, below is an example with speed limits data on the map of Kitchener. See more info about OSM on https://wiki.openstreetmap.org/wiki/Map_features.

After careful investigation of the data, however, we decided that the available road segment dataset on the Waterloo open data portals should suffice for our purposes.

In [ ]:
head(available_features())

In [ ]:
bb_wat <- data_waterloo@bbox 
rownames(bb_wat) <- c("x", "y")
q <-  bb_wat %>% 
  opq() %>% 
  add_osm_feature(key = "maxspeed")  
poi_osm_sp <- osmdata_sp(q)
poi_osm_sp$osm_points <- subset(poi_osm_sp$osm_points, !is.na(poi_osm_sp$osm_points$maxspeed))

In [ ]:
pal <- colorFactor(RColorBrewer::brewer.pal(8, "OrRd"), domain = factor(seq(30,100, 10)))
leaflet() %>% 
  setView(lng = -80.49, lat = 43.43, zoom = 12) %>%
  addTiles() %>%
  addCircleMarkers(lng = coordinates(poi_osm_sp$osm_points)[,1], lat = coordinates(poi_osm_sp$osm_points)[,2], 
                   radius = 10, color = "black", weight = 2, fillColor = ~pal(poi_osm_sp$osm_points$maxspeed), 
                   opacity = 0.8, fillOpacity = 0.8,
                   data = poi_osm_sp) %>% 
  addLegend("bottomleft", pal = pal, values = ~maxspeed, opacity = 1, 
            title = "MaxSpeed (km/h)",
            data = poi_osm_sp$osm_points)

### Exploratory Data Analysis

In [ ]:
data_waterloo %>% 
  leaflet() %>%
  addProviderTiles(providers$CartoDB.Positron) %>%
  addCircleMarkers(stroke = FALSE, fillOpacity = 0.8,  
                   clusterOptions = markerClusterOptions()) %>% 
  addHeatmap(radius = 8)

We checked available variables to understand how to clean and feature engineer them.

In [ ]:
head(data_waterloo@data)

The challenge explicitly asked us to examine the impact of ***day period*** on accidents, and we only found this information in the *light* variable.

In [ ]:
data_waterloo@data %>% count(light)

**Regional roads and highways** with heatmap of traffic collisions in the background.

If you zoom in on regional roads and highways, it seems that collisions were only recorded in intersections. The actual surface of these major roads is probably covered in the dataset - therefore we omit this data during cleaning.

In [ ]:
leaflet() %>%
  addProviderTiles(providers$CartoDB.Positron) %>%
  addPolylines(data = subset(data_waterloo_roads, !is.na(regional_road))) %>% 
  addPolylines(data = subset(data_waterloo_roads, !is.na(highway)), color = "red") %>% 
  addHeatmap(data = data_waterloo, radius = 8, blur = 8)

A good way to understand different elements or categories is to map them and try to understand their role. 

E.g. on the following map we can see that *bulbs* are practically roundabouts in *cul-de-sac*, which are most probably constructed in residential areas.

In [ ]:
pal1 <- colorFactor(c("black", "black", "green", "blue", "orange"), 
                   domain = c("BULB", "HIGHWAY", "RAMP", "ROAD", "ROUNDABOUT"))
leaflet() %>% 
  addTiles() %>%
  addPolylines(data = data_waterloo_roads, color = ~pal1(data_waterloo_roads@data$category))  

We can see that *truck access* is mainly allowed on the major roads (allowed routes are denoted black).

In [ ]:
pal2 <- colorFactor(c("black", "black", "black", "black", "black", "blue"), 
                    domain = c("7-7 MON-FRI", "7-7 MON-SAT", "7-7 MON-SUN", "MON-SUN", "24HR", "NO ACCESS"))
leaflet() %>% 
  addTiles() %>%
  addPolylines(data = data_waterloo_roads,
               color = ~pal2(data_waterloo_roads@data$truck_access)) %>% 
  addHeatmap(data = data_waterloo, radius = 8)

Bus routes seem to be only sporadically entered in the data. Still, we kept it in our analysis as buses can constitute traffic obstructions themselves, and also it seems that bus routes follow a different pattern than the main roads in the hierarchical road network.

In [ ]:
leaflet() %>%                                                             
  addProviderTiles(providers$CartoDB.Positron) %>%
  addPolylines(data = subset(data_waterloo_roads, bus_route == "Y")) %>% 
  addHeatmap(data = data_waterloo, radius = 8)

Numerous classifications are available in the road dataset: *category*, *subcategory*, *carto_class*, *official_road_classification*. After examining their contents, we decided to drop them except for *category* and *subcategory*.

In [ ]:
data_waterloo_roads@data %>% count(category, subcategory, carto_class)

### Data cleaning



In [ ]:
data_waterloo@data <- data_waterloo@data %>% 
  mutate(accidentdate = ymd(accidentdate))
data_waterloo@data$trafficcollisionsid <- NULL
data_waterloo@data <- data_waterloo@data %>% 
  mutate(day_period = case_when(str_detect(light, "Daylight") ~ "Day",
                                str_detect(light, "Dawn") ~ "Dawn",
                                str_detect(light, "Dusk") ~ "Evening",
                                str_detect(light, "Dark") ~ "Night"))
data_waterloo <- subset(data_waterloo, !(is.na(accident_location) | is.na(impact_location) | is.na(environment_condition) | 
                                            is.na(light) ))    

In [ ]:
data_waterloo@data <- data_waterloo@data %>% 
  mutate(fromstreet = str_trim(str_split(location, "@|(btwn)", simplify = TRUE)[,1])) %>%
  mutate(tostreet = str_trim(str_split(location, "@|(btwn)", simplify = TRUE)[,2])) %>% 
  mutate(tostreet_1 = str_trim(str_split(tostreet, "&", simplify = TRUE)[,1])) %>% 
  mutate(tostreet_2 = str_trim(str_split(tostreet, "&", simplify = TRUE)[,2])) %>% 
  dplyr::select(-tostreet)

In [ ]:
data_waterloo_roads <- subset(data_waterloo_roads, (is.na(regional_road) & is.na(highway) ))
cols_to_del <- c("street", "from_street", "to_street", "from_left_address", "from_right_address", "to_left_address", "to_right_address", 
                 "right_side_parity", "regional_road", "highway", "municipality", "general_grid_id",
                 "sequence", "ownership", "plow_route", "median_type", "street_direction", "leaf_route", "left_municipality", 
                 "right_municipality", "create_by", "update_date", "update_by", "source", "revision_notes", "status", "row_width",
                 "quarter_grid_id", "wardid", "planningcommunityid", "str_roadsegmentid", "fireresponsezoneid", "parcelid",
                 "shoulder", "maintenance", "carto_class", "operations_class", "salt_sand_route", "startx", "starty", "endx", "endy", 
                 "road_patrol", "operations_mto_class", "last_road_patrol_date", 
                 "next_road_patrol_date", "surface_layer_type", "official_road_classification", "aadt_year", "aadt_type", 
                 "calculated_mto_class", "service_year", "next_road_patrol_due")
data_waterloo_roads@data[cols_to_del] <- NULL
data_waterloo_roads@data <- data_waterloo_roads@data %>% 
  mutate(flow_direction = if_else(flow_direction == "TwoWay", 1, 0)) %>% 
  mutate(aadt = log(aadt+1)) %>% 
  mutate(lanes = if_else(lanes >= 5, 5L, .$lanes)) %>% 
  mutate(speed_limit_km = case_when(speed_limit_km == 50 ~ "50",
                                 speed_limit_km > 50 ~ "50+",
                                 speed_limit_km < 50 ~ "50-")) %>% 
  mutate(truck_access = if_else(truck_access == "NO ACCESS", 0, 1)) %>% 
  mutate(bus_route = if_else(bus_route == "Y", 1, 0)) %>% 
  mutate(shape__length = log(shape__length))

To perform **spatial join** of the 2 datasets (collisions and roads), we tried to match the spatial points of collisions to the polylines of the road segments, but only a small part of the connected features could be really captured this way. Therefore we decided to establish a **10 m wide buffer** (1/11114 planar coordinates) along the road segments and do the spatial join on the resulting polygons. We experimented with wider buffers as well, but this seemed to be a good compromise between picking up the points close to the road segments, but not picking up points already belonging to other segments as well.

Of course the spatial join is a delicate task and could have been refined more, and also the points still left out from the joined dataset could be grabbed (check out the red points not on blue buffer areas in the map below when zooming in the map), e.g. by their minimal distance to any road segments, but we decided to keep this simple algorithm for this demonstration task.

In [ ]:
data_waterloo_roads_buffer <- rgeos::gBuffer(data_waterloo_roads, byid = TRUE, width = 1/11114,
                                             capStyle = "FLAT", joinStyle = "MITRE")  # 10 m wide 

In [ ]:
leaflet(data_waterloo_roads_buffer, options = leafletOptions(minZoom = 14)) %>%
  addProviderTiles(providers$CartoDB.Positron) %>%
  addPolygons() %>% 
  addPolylines(data = data_waterloo_roads) %>% 
  addCircles(data = data_waterloo, radius = 10, color = "red")


## Data preparation for the ML exercise

**We decided to include summary data of all the accidents that belong to a road segment, and thus investigate the factors explaining why a given road segment is more dangerous (in terms of more traffic accidents) than another.**

We constructed 2 target features:

- the number of accidents in a given road segment throughout the whole period (2005-2018)
- introducing a categorical variable for road segments with 0, 1-3 and 4+ accidents in the whole period

Also constructed the same targets by the 4 day period (dawn, day, evening, night) to be able to examine differences from this ascpect. 

*Note that for lack of time we have not worked with all potential factors, e.g. artificial light augmenting conditions for drivers relative to natural light.*

In [ ]:
data_waterloo_joined4 <- over(data_waterloo_roads_buffer, data_waterloo, returnList = TRUE) # list of DFs output 
data_waterloo_joined4_all <- data_waterloo_joined4 %>% map_df(rbind)       # 15853 rows
num_accidents <- sapply(data_waterloo_joined4, nrow)

In [ ]:
paste2 <- function(x, y, sep = "") paste(x, y, sep = sep)  

col_collisiontype <- data_waterloo_joined4 %>% 
  map_chr(~ reduce(.x$collision_type, paste2, .init = ""))
col_collisiontype <- if_else(col_collisiontype == "", NA_character_, col_collisiontype)
col_collision_type_int <- if_else(is.na(col_collisiontype), 0L, str_count(col_collisiontype, "INTERSECTION") )
col_collision_type_mid <- if_else(is.na(col_collisiontype), 0L, str_count(col_collisiontype, "MIDBLOCK") )

In [ ]:
col_dayperiod <- data_waterloo_joined4 %>% 
  map_chr(~ reduce(.x$day_period, paste2, .init = ""))
col_dayperiod <- if_else(col_dayperiod == "", NA_character_, col_dayperiod)
col_dayperiod_day <- if_else(is.na(col_dayperiod), 0L, str_count(col_dayperiod, "Day") )
col_dayperiod_nig <- if_else(is.na(col_dayperiod), 0L, str_count(col_dayperiod, "Night") )
col_dayperiod_daw <- if_else(is.na(col_dayperiod), 0L, str_count(col_dayperiod, "Dawn") )
col_dayperiod_eve <- if_else(is.na(col_dayperiod), 0L, str_count(col_dayperiod, "Evening") )

In [ ]:
data_waterloo_ML <- spCbind(data_waterloo_roads, num_accidents)
data_waterloo_ML <- spCbind(data_waterloo_ML, col_dayperiod_day)
data_waterloo_ML <- spCbind(data_waterloo_ML, col_dayperiod_nig)
data_waterloo_ML <- spCbind(data_waterloo_ML, col_dayperiod_daw)
data_waterloo_ML <- spCbind(data_waterloo_ML, col_dayperiod_eve)
data_waterloo_ML <- spCbind(data_waterloo_ML, col_collision_type_int)
data_waterloo_ML <- spCbind(data_waterloo_ML, col_collision_type_mid)

This is the final dataset for the ML exercise:

In [ ]:
skimr::skim_without_charts(data_waterloo_ML@data) %>%
  dplyr::select(-complete_rate)

In [ ]:
data_ml <- data_waterloo_ML@data
setDT(data_ml)

data_ml[num_accidents == 0, num_accidents_cat:= "noAccident"]
data_ml[num_accidents > 0 & num_accidents <= 3, num_accidents_cat:= "lowAccident"]
data_ml[num_accidents > 3, num_accidents_cat:= "highAccident"]

data_ml[, street_type:= as.factor(street_type)]
data_ml[is.na(street_type), street_type:= "other"]
data_ml[is.na(plow_priority), plow_priority:= -1]
data_ml[, category:= as.factor(category)]
data_ml[, subcategory:= as.factor(subcategory)]
data_ml[is.na(pavement_width), pavement_width:= -1]
data_ml[is.na(flow_direction), flow_direction:= 1]
data_ml[, speed_limit_km:= as.factor(speed_limit_km)]
data_ml[is.na(truck_access), truck_access:= 0]
data_ml[is.na(speed_limit_km), speed_limit_km:= "50"]
data_ml[speed_limit_km=="50+", speed_limit_km:= "50"]
data_ml[is.na(bus_route), bus_route:= 0]
data_ml[is.na(shape__length), pavement_width:= -1]
data_ml[is.na(aadt), aadt:= -1]

Benchmark for the categorical variable based on statistical distribution (probability). This helps to see if our model performs better than prediction based on knowing the ratio of the different categories.

In [ ]:
dist_num_acc <- summary(factor(data_ml$num_accidents_cat))
data_ml[, num_accidents_cat_bm:= sample(c("highAccident", "lowAccident", "noAccident"), prob = c(dist_num_acc['highAccident'],
                                                                                                 dist_num_acc['lowAccident'],
                                                                                                 dist_num_acc['noAccident']),
                                        size = nrow(data_ml), replace = T)]

### EDA for the ML target variables

Exploring the target variables, e.g. their basic histograms both in normal&log and the categorical forms.

*Note that the variables reveals that we are dealing with an unbalanced dataset. We have not addressed this issue for lack of time.*

In [ ]:
options(repr.plot.width=7, repr.plot.height=1.5)
plot1 <- ggplot(data_ml, aes(x = num_accidents)) +
  geom_histogram() +
  theme_bw()
plot2 <- ggplot(data_ml, aes(x = log1p(num_accidents))) +
  geom_histogram() +
  theme_bw()
plot2 <- ggplot(data_ml, aes(x = log1p(num_accidents))) +
  geom_histogram() +
  theme_bw()
plot3 <- ggplot(data_ml, aes(x = num_accidents_cat)) +
  geom_bar() +
  theme_bw() +
  coord_flip()
grid.arrange(plot1, plot2, plot3, ncol=3, widths = c(30,30,40))

A few more visualizations of the relationship of the target and independent variables.

Interestingly, there seem to be more accidents on average in roads with speed limits (lower than 50 km/h). This can have various explanations that speed limits were imposed on the already more dangerous road segments (maybe during the long period investigated). Also, we don't know anything about the severity of these accidents, and minor accidents with low speeds in residential areas obviously have lower social costs than severe accidents with higher speeds on major roads.

In [ ]:
options(repr.plot.width=5, repr.plot.height=1.5)
data_ml[, speed_limit_km:= as.factor(speed_limit_km)]
data_ml[is.na(speed_limit_km), speed_limit_km:= "50"]
ggplot(data_ml, aes(x = as.factor(speed_limit_km), y = log1p(num_accidents))) +
  geom_boxplot() +
  theme_bw()

Average annual daily turnover (aadt) is also included in the roads dataset. Unfortunately this data is provided in different years for each road segment (there is a variable Aadt_Year), so it is not very useful for comparison, but this is the best we have. We use it in log form.

In [ ]:
options(repr.plot.width=7, repr.plot.height=1.5)
plot1 <- ggplot(data_ml, aes(x = (aadt))) +
  geom_histogram() +
  theme_bw()
plot2 <- ggplot(data_ml, aes(x = (aadt), y = log1p(num_accidents))) +
  geom_jitter() +
  geom_smooth() +
  theme_bw()
grid.arrange(plot1, plot2, ncol=2)

## Model building
### Numerical target

We separated 20% of the data for testing.

In [ ]:
set.seed(1234)
trainIndex <- createDataPartition(data_ml$num_accidents, p = 0.8, list = FALSE, times = 1)
dataTrain <- data_ml[ trainIndex,]
dataTest  <- data_ml[-trainIndex,]

Calculating benchmark metrics for the models.

In [ ]:
# For normal values
cat("Mean Average Error of benchmark: ", MAE(obs = dataTrain$num_accidents, pred = mean(dataTrain$num_accidents)), "\n")
cat("Root Mean Squared Error of benchmark: ", RMSE(obs = dataTrain$num_accidents, pred = mean(dataTrain$num_accidents)), 
    "\n\n")

# For category values
conf_matrix_bm <- table(dataTrain$num_accidents_cat, dataTrain$num_accidents_cat_bm)
cat("Benchmark confusion matrix:")
conf_matrix_bm
accuracy_bm <- (conf_matrix_bm[1, 1] + conf_matrix_bm[2, 2] + conf_matrix_bm[3, 3])/ sum(conf_matrix_bm)
cat("Benchmark accuracy (%): ", accuracy_bm)

In [ ]:
control <- trainControl(method='repeatedcv', 
                        number=10,
                        repeats=5,
                        verboseIter = FALSE)

In [ ]:
set.seed(1234)
accident_lasso_model <- train(as.numeric(num_accidents) ~ lanes + street_type + plow_priority  + category  +
                             subcategory + pavement_width + flow_direction + speed_limit_km  + truck_access +
                             bus_route + shape__length + aadt + poly(aadt, 3),
                             data=dataTrain,
                             method='glmnet',
                             #tuneLength = 5,
                             tuneGrid=expand.grid(alpha=1, lambda=seq(0, 0.06, by = 0.005)),
                             trControl = control,
                             metric='RMSE',
                             importance = "impurity")

In [ ]:
accident_lasso_model

In [ ]:
rfGrid <- expand.grid(mtry = c(10, 15),
                      min.node.size = c(10),
                      splitrule = c("extratrees"))
set.seed(1234)
accident_rf_model <- train(as.numeric(num_accidents) ~ lanes + street_type + plow_priority  + category  +
                             subcategory + pavement_width + flow_direction  + speed_limit_km  + truck_access +
                             bus_route + shape__length + aadt,
                             data=dataTrain,
                             method='ranger',
                             tuneGrid = rfGrid,
                             trControl = control,
                             metric='RMSE',
                             importance = "impurity")

The cross-validated RMSE improved to 8.3 using a Random Forest algorithm, which is only a limited improvement.

In [ ]:
accident_rf_model

There is a great variance in the individual fits of the model, unfortunately the different samples of CV do not behave consistenlty.

In [ ]:
accident_rf_model$resample

The RF model performed better on the train set, so we check it on the test set. Results are not too far from the train's.

In [ ]:
dataTest[, predicted_accident:= predict(accident_rf_model, newdata = dataTest)]
MAE(dataTest$predicted_accident, dataTest$num_accidents)
RMSE(dataTest$predicted_accident, dataTest$num_accidents)

These variables explain a greater part of the target's variance. It is not surprising that annual average daily traffic and the length of the road segment are important.

In [ ]:
options(repr.plot.width=4.5, repr.plot.height=3.5)
plot(varImp(accident_rf_model, scale = FALSE), 15)

High number of accidents are hard to capture by the models.

In [ ]:
options(repr.plot.width=3.5, repr.plot.height=3.5)
### Predict on the train set:
dataTrain[, predicted_accident:= predict(accident_rf_model, newdata = dataTrain)]

ggplot(dataTrain, aes(x = num_accidents, y = predicted_accident)) +
  geom_point(alpha = .2) +
  geom_abline(slope = 1) +
  geom_smooth() +
  theme_bw() +
  coord_fixed(ratio = 1, xlim = c(0, 150), ylim = c(0, 150))

In [ ]:
data_ml[, predicted_accident:= predict(accident_rf_model, newdata = data_ml)]
#MAE(data_ml$predicted_accident, data_ml$num_accidents)
cat("RMSE on full data: ", RMSE(data_ml$predicted_accident, data_ml$num_accidents))

ggplot(data_ml, aes(x = log1p(num_accidents), y = log1p(predicted_accident))) +
  geom_jitter(width = c(0.05)) +
  geom_abline(slope = 1) +
  geom_smooth() +
  theme_bw()

### Categorical target

In [ ]:
control <- trainControl(method='repeatedcv', 
                        number=10,
                        repeats=3,
                        verboseIter = FALSE,
                        classProbs = TRUE)

In [ ]:
set.seed(1234)
accident_cat_lasso_model <- train(as.factor(num_accidents_cat) ~ lanes + street_type + plow_priority  + category  +
                             subcategory +  pavement_width + flow_direction  + speed_limit_km  + truck_access + bus_route +
                               shape__length + aadt,
                           data=dataTrain,
                           method='glmnet',
                           #tuneLength = 10,
                           tuneGrid=expand.grid(alpha=1, lambda=seq(0.005, 0.015, by = 0.005)),
                           trControl = control,
                           importance = "impurity")

In [ ]:
accident_cat_lasso_model

In [ ]:
rfGrid <- expand.grid(mtry = c(10),
                      min.node.size = c(10),
                      splitrule = c("gini"))
set.seed(1234)
accident_cat_rf_model <- train(as.factor(num_accidents_cat) ~ lanes + street_type + plow_priority  + category  +
                           subcategory +  pavement_width + flow_direction  + speed_limit_km  + truck_access + bus_route +
                           shape__length + aadt,
                           data=dataTrain,
                           method='ranger',
                           tuneGrid = rfGrid,
                           trControl = control,
                           importance = "impurity")

In [ ]:
accident_cat_rf_model

We choose the RF model.

In [ ]:
dataTest[, predicted_accident_cat:= predict(accident_cat_rf_model, newdata = dataTest)]
conf_matrix_test <- table(dataTest$num_accidents_cat, dataTest$predicted_accident_cat)
conf_matrix_test

In [ ]:
accuracy_test <- (conf_matrix_test[1, 1] + conf_matrix_test[2, 2] + conf_matrix_test[3, 3])/ sum(conf_matrix_test)
accuracy_test

Accuracy on test set is almost the same as on the train set, so the model, although of limited performance, it generalizes well.

In [ ]:
## Using the model on the original dataset
data_ml[, predicted_accident_cat:= predict(accident_cat_rf_model, newdata = data_ml)]
conf_matrix_all <- table(data_ml$num_accidents_cat, data_ml$predicted_accident_cat)
conf_matrix_all

In [ ]:
accuracy_all <- (conf_matrix_all[1, 1] + conf_matrix_all[2, 2] + conf_matrix_all[3, 3])/ sum(conf_matrix_all)
accuracy_all

Using the model on the original dataset:

In [ ]:
options(repr.plot.width=3.5, repr.plot.height=3.5)
plot(varImp(accident_cat_rf_model, scale = FALSE), 15)

In [ ]:
models_default <- list(lasso_cat = accident_cat_lasso_model,
                       rf_cat = accident_cat_rf_model)
models_resampling <- resamples(models_default)

summary(models_resampling, metric = "Accuracy")
options(repr.plot.width=2.5, repr.plot.height=2.5)
bwplot(models_resampling, metric= "Accuracy")

### Demonstrating predictions on map

In [ ]:
# introducing predictions to the spatial DF
data_waterloo_ML@data$predicted_accident  <- data_ml$predicted_accident
data_waterloo_ML@data$predicted_accident_cat  <- data_ml$predicted_accident_cat

In [ ]:
pal3 <- colorFactor(c("red", "orange", "green"), 
                    domain = factor(c("highAccident", "lowAccident", "noAccident")))
leaflet() %>% 
  addTiles() %>%
  addPolylines(data = data_waterloo_ML,
               color = ~pal3(data_waterloo_ML@data$predicted_accident_cat)) %>% 
  addLegend("bottomleft", pal = pal3, values = ~factor(predicted_accident_cat), opacity = 1, 
            title = "Road segments with predicted accident category",
            data = data_waterloo_ML@data)

### Summary of findings

Our decision to choose road segments and the number of accidents on them as he basis of analysis limited the information we could consider in our models (e.g. actual weather conditions of the accidents could not be part of this data). 

From a smart city perspective, however, we think that this approach is useful, as it may help to reveal road segments that are more dangerous (in terms of more traffic accidents) than another.

*We also carried out the analysis using the 4 available day periods (dawn, day, evening, night) separately, to see if patterns are different in any period. Unfortunately we are running out of time so cannot incorporate it into this notebook.*

## Conclusions

### Scalability and Impact
Tell us how applicable and scalable your solution is if you were to implement it in a city. Identify possible limitations and measure the potential social impact of your solution.

The current dataset is a nice start but not sufficient for a comprehensive traffic management improvement strategy. 
Key limitations:
 - Data on the severity of accidents is missing, so this model doesn't allow to optimize to avoid for example fatalities or major incidents.
 - The current dataset is limited to car accidents, we don't learn anything about bicyle and pedestrian accidents.
 - Due to the lack of time we were also not able to build a model complex enough to handle weather data - while weather (especially snow and rain) should logically be a major factor for accident prediction.
 - Our dataset doesnt' have information on demographics, so in it's current form, we wouldn't be able to use to target certain groups (for example: taxi drivers, senior or junior drivers) with campains or education.
 
 
The heatmap part of our solution should be possible to implement for any city since it visualizes already existing data. It could be used to alert drivers for dangerous spots. 

The current model can be applied to all cities with similar level of data. As we assume that police databases have data of this level, after processing these (which is not a small task!) the model could be applied to other cities.

This model would be even more useful if it would allow to focus on traffic accidents involving other types of transportation (bikes, pedestrians) and on avoiding fatalities. 


### Future Work
Now picture the following scenario: imagine you could have access to any type of data that could help you solve this challenge even better. What would that data be and how would it improve your solution? 🚀

An ideal dataset would contain data on the following factors:


 - Road Data - Our dataset has road priority data available, however the regional roads appear to be missing from the dataset. The slope grade of each road would allow a finer prediction of risk. 


 - Traffic Data - Traffic flow would help to predict accidents, as it is an crucial factor for planning traffic safety and efficiency. Tolls, parking fees, and perhaps the locations of traffic signs and lights would allow a better finetuning of the traffic flow. 

    Close to live traffic data is possible via traffic sensors on main roads and via GPS senders in buses, this would also allow a faster reaction to traffic congestions. 
    

 - User Behaviour Data - our current dataset didn't have any information police would have available, but we are certain this information is available for cases where the police was involved: 
 Alcohol and drug use, other passengers, personal injuries and damage (car or collateral), speeding habits
Vehicle information (type, age)


 - Data on other transportation methods: bicycles and pedestrians are also part of the traffic.
Public transportation as well, however we may assume this is already included in the dataset without added flag.

A more granular time data (hourly split) would be also a much finer prediction.